In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from pprint import pprint
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
df = pd.read_csv("lsh_assignment_data.csv")

In [3]:
# importing stop word 

import nltk
# nltk.download("stopwords")
from nltk.corpus import stopwords 

stopwords = stopwords.words('english')
# print(stopwords)


In [4]:
# split data into Trainng and Test data

train_data = df.iloc[:-10,:]
test_data = df.iloc[-10:,:]

In [5]:
# tfidf-vectorizer with bi-gram and tri-gram of words having min-frequency of 10 and only consider 4000 features


vectorizer = TfidfVectorizer(ngram_range=(2,3), min_df=10, max_features=4000)#, stop_words=stopwords)
n_grams = vectorizer.fit_transform(train_data["text"])

In [8]:
n_grams.shape

(2215, 4000)

In [9]:
# create random planes 
# set the random seed to 0
np.random.seed(0)

def create_random_plane(a, dim):
    w = []
    for i in range(1,a+1):
        w.append(np.random.normal(0,1,dim)) # the dimension of the each plane will be same as dimension as the each data point in train_data
    return w
    

In [10]:
m_planes = create_random_plane(5, n_grams.shape[1])

In [11]:
def hash_val(vector, m_planes):
    '''
    This function return the hash values by computing the dot product of vector and m_planes
    vector and m_planes belongs to d-dimension then hash values will "d" number of char
    
    # if (w.T).(v) < 0 --> 0
    # if (w.T).(v) > 1 --> 1
    '''
    st = ''
    for i in m_planes:
        dot_pr = (vector.dot(i))  
        ## as type(vector) -->  scipy.sparse.csr.csr_matrix
        ## type(m_planes[i]) --> np.ndarray
        if dot_pr < 0:
            st += "0"
        elif dot_pr >0:
            st+="1"
    return st
            

In [14]:
hash_val(vec1, m_planes)

'01101'

In [15]:
vec_label = list(range(len(list(train_data["category"]))))

In [17]:
def hash_create_2(n_grams, vector_label, m_planes):
    '''
    input:
            vectors ---> vector representation of each data point
            vector_label --> vector name of corresponding vector-form in vectors
            m_planes ---> these are the randomnly generated planes 
    
    I will create a dict of the given vectors,and store them as hash_value:vectore_name
    '''
    hash_dict = {}
    
    for i in tqdm(range(len(vector_label))):
        hash_value = hash_val(n_grams[i], m_planes)
        hash_dict.setdefault(hash_value, []).extend([vector_label[i]])
    return hash_dict

In [18]:
hash_dict = hash_create_2(n_grams, vec_label, m_planes)

100%|████████████████████████████████████████████████████████████████████████████| 2215/2215 [00:00<00:00, 10806.31it/s]


# TEST

In [19]:
test_data

,category,text
2215,NaN,junk e-mails on relentless rise spam traffic i...
2216,NaN,top stars join us tsunami tv show brad pitt r...
2217,NaN,rings of steel combat net attacks gambling is ...
2218,NaN,davies favours gloucester future wales hooker ...
2219,NaN,beijingers fume over parking fees choking traf...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...
2224,NaN,souness delight at euro progress boss graeme s...


In [20]:
test_gram = vectorizer.transform(test_data.text).toarray()

In [22]:
def cosine_sim(vector1, vector2):
    '''
    The function return the cosime similarity between vector1 and vector2
    '''
    return (vector2.dot(vector1))/(np.linalg.norm(vector1)* np.linalg.norm(vector2))

In [23]:
test_hash = []
for i in test_gram:
    test_hash.append(hash_val(i, m_planes))

test_hash

['01110',
 '10110',
 '01001',
 '01101',
 '10110',
 '00110',
 '01010',
 '10011',
 '10001',
 '10010']

In [24]:
new = []
# count = 0
for i in test_gram:
    new.append([x for x in hash_dict[hash_val(i, m_planes)]])

    
print(new[0]) 
## this dict contain all the vector_name that have same hash_values as the query_points

[23, 96, 105, 153, 239, 243, 315, 401, 405, 468, 511, 587, 680, 694, 771, 780, 809, 847, 855, 876, 963, 973, 1029, 1113, 1175, 1223, 1224, 1378, 1408, 1416, 1461, 1481, 1576, 1665, 1691, 1899, 1930, 2022, 2037, 2171, 2213]


In [26]:
train_vect = n_grams.toarray() # convert the n_gram(spare csr.matrix) to numpy.ndarray, for easy calculation


knn_11 = []
for i in range(0,len(new)):
    cosine_similarity = {}
    for j in range(len(train_vect[new[i]])):
        cos_sim = cosine_sim(test_gram[i], train_vect[new[i][j]]) 
        ## computing the cosine-sim between all the querry_points with the training points(which have same hash_value as the querry point)
        cosine_similarity[new[i][j]] = cos_sim 
        ## then store the value in dict PT_NAME:cosine-sim(between xq and PT_NAME)
    knn_11.append(cosine_similarity) 
    ## Finally append all the dicts in the list, which contain consine-sim between xq and the pts which have same hash_value as xq

In [33]:
cc = sorted(((value, key) for (key,value) in knn_11[0].items()), reverse=True)[:10]
cc

[(1.0000000000000002, 963),
 (0.11124613300644823, 1416),
 (0.10942722899280966, 239),
 (0.09309289878365656, 2213),
 (0.09309289878365656, 401),
 (0.08702680027698745, 2022),
 (0.06702712530502797, 315),
 (0.05782779241516406, 105),
 (0.056929721379619776, 2037),
 (0.04513601479561055, 1408)]

In [29]:
c_f = list(v for k,v in cc)
c_f

[963, 1416, 239, 2213, 401, 2022, 315, 105, 2037, 1408]

In [30]:
from collections import Counter
 
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]


## source -- https://www.geeksforgeeks.org/python-find-most-frequent-element-in-a-list/

In [31]:
pred_class_labels = []
for i in range(len(knn_11)):
    ## this line sort the dict (which contain all the pts which have hash_value same as query point along with their cosine-sim between xq and that point)
    ## the output of this is list, which have point in descending order of cosine-sim (between xq and pt)
    top_11_nss = (list(v for k,v in (sorted(((value, key) for (key,value) in knn_11[i].items()), reverse=True)[:10])))
    # print(top_11_nss)
    
    class_label = [ train_data.category.iloc[i] for i in top_11_nss] ## find all the labels of the top 11-NN for each querry point
    # print(class_label)
    pred_class_labels.append(most_frequent(class_label))
    
    
print(pred_class_labels) 

['tech', 'entertainment', 'tech', 'tech', 'business', 'business', 'politics', 'entertainment', 'politics', 'sport']


In [32]:
x= ['tech', 'entertainment', 'tech', 'sport', 'business', 'business', 'politics', 'entertainment', 'politics', 'sport']
print(x)



['tech', 'entertainment', 'tech', 'sport', 'business', 'business', 'politics', 'entertainment', 'politics', 'sport']


## Readings/References 

    https://www.pinecone.io/learn/locality-sensitive-hashing-random-projection/
    https://santhoshhari.github.io/Locality-Sensitive-Hashing/
    

In [ ]:
###########################################
## GRADER CELL: Do NOT Change this.
# This cell will print "Success" if your implmentation of the predictLabels() is correct and the accuracy obtained is above 80%.
# Else, it will print "Failed"
###########################################
import numpy as np

# Predict the labels using the predictLabels() function
Y_custom = np.array(predictLabels(test_data))

# Reference grader array - DO NOT MODIFY IT
Y_grader = np.array(['tech', 'entertainment', 'tech', 'sport', 'business', 'business', 'politics', 'entertainment', 'politics', 'sport'])

# Calculating accuracy by comparing Y_grader and Y_custom
accuracy = np.sum(Y_grader==Y_custom) * 10

if accuracy >= 80:
    print("******** Success ********","Accuracy Achieved = ", accuracy,'%')
else:
    print("####### Failed #######","Accuracy Achieved = ", accuracy,'%')
    print("\nY_grader = \n\n", Y_grader)
    print("\n","*"*50)
    print("\nY_custom = \n\n", Y_custom)
